In [11]:
tmp <-read.table('handovercompiled.Yanis.080919.csv',sep=",",header=T)
rownames(tmp) <- tmp$data_pd
master <- read.table('../../../data/initial_dataset/Master_04_10_2019.csv',sep=",",header=T)
rownames(master) <- master$data_pd

In [17]:
colnames(tmp)

[1] "donor_id"                "data_pd"                
 [3] "sample_pd.x"             "date_entry"             
 [5] "trtcode"                 "AMLID"                  
 [7] "dead60"                  "MRDgp1"                 
 [9] "MRDgp2"                  "CRby1"                  
[11] "CRby2"                   "MRD1"                   
[13] "MRD2"                    "LAIPdetection1"         
[15] "LAIPvalue1"              "Sample1"                
[17] "DiagnosticLAIPscreening" "LAIPdetection2"         
[19] "LAIPvalue2"              "Sample2"                
[21] "LAIPassessed1"           "LAIPassessed2"          
[23] "trial.y"                 "sample_pd.y"            
[25] "OS_randomization"        "DEADStatus"             
[27] "RFSyears"                "RFSStatus"              
[29] "RelapseCIStatus"         "DaystoFirstCR"          
[31] "CODCause"                "OS_CR"                  
[33] "CRdate"                  "randDate_PB"            
[35] "CRDate_PB"               "relapseDate_PB"         
[37] "boneDate_PB"             "longCR_PB"

In [55]:
colnames(master)

[1] "X"                       "donor_id"               
  [3] "data_pd"                 "sample_pd.x"            
  [5] "ASXL1"                   "ASXL2"                  
  [7] "ASXL3"                   "ATRX"                   
  [9] "BAGE3"                   "BCOR"                   
 [11] "BRAF"                    "CBFB"                   
 [13] "CBL"                     "CDKN2A"                 
 [15] "CEBPA_bi"                "CEBPA_mono"             
 [17] "CNTN5"                   "CREBBP"                 
 [19] "CSF1R"                   "CSF3R"                  
 [21] "CTCF"                    "CUL2"                   
 [23] "CUX1"                    "DNMT3A"                 
 [25] "EED"                     "ETV6"                   
 [27] "EZH2"                    "FBXW7"                  
 [29] "FLT3_ITD"                "FLT3_TKD"               
 [31] "FLT3_other"              "GATA1"                  
 [33] "GATA2"                   "GNAS"                   
 [35] "GNB1"                    "IDH1"                   
 [37] "IDH2_p.R140"             "IDH2_p.R172"            
 [39] "JAK2"                    "JAK3"                   
 [41] "KANSL1"                  "KDM6A"                  
 [43] "KIT"                     "KMT2C"                  
 [45] "KMT2D"                   "KMT2E"                  
 [47] "KRAS"                    "LUC7L2"                 
 [49] "MED12"                   "MLL"                    
 [51] "MPL"                     "MYC"                    
 [53] "NF1"                     "NFE2"                   
 [55] "NOTCH1"                  "NPM1"                   
 [57] "NRAS_other"              "NRAS_p.G12_13"          
 [59] "NRAS_p.Q61_62"           "PDS5B"                  
 [61] "PHF6"                    "PPFIA2"                 
 [63] "PRPF8"                   "PTEN"                   
 [65] "PTPN11"                  "PTPRF"                  
 [67] "PTPRT"                   "RAD21"                  
 [69] "RIT1"                    "RUNX1"                  
 [71] "S100B"                   "SETBP1"                 
 [73] "SF1"                     "SF3B1"                  
 [75] "SMC1A"                   "SMC3"                   
 [77] "SMG1"                    "SPP1"                   
 [79] "SRSF2"                   "STAG2"                  
 [81] "STAT5B"                  "SUZ12"                  
 [83] "TET2"                    "TP53"                   
 [85] "U2AF1"                   "WT1"                    
 [87] "ZRSR2"                   "trial.x"                
 [89] "date_entry"              "gender"                 
 [91] "age"                     "ethnic"                 
 [93] "ahd"                     "perf_status"            
 [95] "secondary"               "wbc"                    
 [97] "hb"                      "plt"                    
 [99] "bm_blasts"               "neut"                   
[101] "intense"                 "trtcode"                
[103] "acmarr"                  "acmcar"                 
[105] "acmhvd"                  "acmibd"                 
[107] "acmdia"                  "acmcvd"                 
[109] "acmpsy"                  "acmobe"                 
[111] "acminf"                  "acmrhe"                 
[113] "acmpep"                  "acmren"                 
[115] "acmpulmod"               "acmpulsev"              
[117] "acmpri"                  "acmhepmil"              
[119] "acmhepsev"               "acmoth"                 
[121] "cod"                     "cr"                     
[123] "os"                      "os_status"              
[125] "pbmts"                   "pbmty"                  
[127] "t_v_11"                  "add_10p"                
[129] "add_10q"                 "add_11p"                
[131] "add_11q"                 "add_12p"                
[133] "add_12q"                 "add_13p"                
[135] "add_13q"                 "add_14p"                
[137] "add_14q"                 "add_15p"                


In [47]:
for (col in c(1:38)){
    cat(colnames(merge)[col],(all.equal(merge[,col],merge[,col+38])))
    print(" ")
    
}


donor_id.x Attributes: < Component “levels”: Lengths (1985, 2428) differ (string compare on first 1985) > Attributes: < Component “levels”: 1985 string mismatches >[1] " "
data_pd.x Attributes: < Component “levels”: Lengths (1985, 2428) differ (string compare on first 1985) > Attributes: < Component “levels”: 1907 string mismatches >[1] " "
sample_pd.x.x Attributes: < Component “levels”: Lengths (1985, 2428) differ (string compare on first 1985) > Attributes: < Component “levels”: 1907 string mismatches >[1] " "
date_entry.x Attributes: < Component “levels”: Lengths (1132, 1343) differ (string compare on first 1132) > Attributes: < Component “levels”: 1132 string mismatches > 1985 string mismatches[1] " "
trtcode.x TRUE[1] " "
AMLID.x TRUE[1] " "
dead60.x TRUE[1] " "
MRDgp1.x TRUE[1] " "
MRDgp2.x TRUE[1] " "
CRby1.x TRUE[1] " "
CRby2.x TRUE[1] " "
MRD1.x TRUE[1] " "
MRD2.x TRUE[1] " "
LAIPdetection1.x TRUE[1] " "
LAIPvalue1.x TRUE[1] " "
Sample1.x TRUE[1] " "
DiagnosticLAIPscreening.x 

In [37]:
identical(merge[['donor_id.x']],merge[['donor_id.y']])

[1] FALSE

In [36]:
merge[,c('donor_id.x','donor_id.y')] <- as.character(merge[,c('donor_id.x','donor_id.y')])

In [43]:
head(merge)

donor_id.x,data_pd.x,sample_pd.x.x,date_entry.x,trtcode.x,AMLID.x,dead60.x,MRDgp1.x,MRDgp2.x,CRby1.x,⋯,RelapseCIStatus.y,DaystoFirstCR.y,CODCause.y,OS_CR.y,CRdate.y,randDate_PB.y,CRDate_PB.y,relapseDate_PB.y,boneDate_PB.y,longCR_PB.y
16-5631,PD14865a,PD14865a,2/22/2010,160006,NA,NA,NA,NA,NA,⋯,2,NA,INFECTION,NA,NA,0,0,0,0,0
16-5645,PD14866a,PD14866a,3/16/2010,160006,NA,NA,NA,NA,NA,⋯,1,72,RECURRENT LEUK,1.6208077,2010-05-27,0,0,0,0,0
16-5665,PD14867a,PD14867a,4/20/2010,160005,NA,NA,NA,NA,NA,⋯,2,NA,RESISTANT DIS,NA,NA,0,0,0,0,0
16-5697,PD14868a,PD14868a,4/6/2010,160005,NA,NA,NA,NA,NA,⋯,2,NA,RESISTANT DIS,NA,NA,0,0,0,0,0
16-2203,PD14869c,PD14869c,3/6/2010,160002,16-2203,0,0,1,Yes,⋯,0,34,NOT DEAD,4.6132786,2010-07-07,0,0,0,0,0
16-5731,PD14871a,PD14871a,8/16/2010,160005,NA,NA,NA,NA,NA,⋯,1,60,INFECTION,0.2135524,2010-10-15,0,0,0,0,0
